In [2]:
import sys
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import os
import cv2
from torch.utils.data import DataLoader, TensorDataset, Dataset

%load_ext autoreload
%autoreload 2
# %matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES']='0,2'
sys.path.append("/home/shiyi/gpu/")

from gpu_allocation import set_gpu
num_gpu = 1
set_gpu(num_gpu)

Found 8 GPU(s) only 4 gpu below threshold
Using GPU 1


In [9]:
#hyperparameters 

IMG_SIZE = 299
BATCH_SIZE = 128
CLASSES = 8
EPOCH = 200

IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LR = 0.001

In [10]:
#dir and path

csv_path = '/data/AlgProj/tct_yaoms/data/tct_0702_preprocess_iou0.1_img_series/iou0.1dataaumentation.csv'
csv = pd.read_csv(csv_path)
csv = csv.loc[csv['label']!=1].reset_index(drop=True)
len(csv)

3457

In [11]:
#train val transforms definition

def train_test_split(df, test_size, shuffle=True):
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)
    #split
    length = len(df)
    threspoint = int((1 - test_size)*length)
    train_df = df.loc[:threspoint-1,:]
    test_df = df.loc[threspoint:,:]
    
    d = {}
    d['train'] = train_df
    d['test'] = test_df.reset_index(drop=True)
    return d

train_transformer = transforms.Compose([transforms.Resize((336,336)),
                                 transforms.RandomRotation((-180, 180)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomResizedCrop((299,299)),  
                                 transforms.ToTensor(),
                                       transforms.Normalize(IMG_MEAN,IMG_STD)]
                                )
test_transformer = transforms.Compose([transforms.Resize((336,336)),
                                  transforms.CenterCrop((299,299)),
                                 transforms.ToTensor(),
                                      transforms.Normalize(IMG_MEAN,IMG_STD)])
train_test_transformer = {'train':train_transformer, 'test':test_transformer}

In [12]:
train_test_dict = train_test_split(csv, 0.2)
len(train_test_dict['test'])

692

In [13]:
class TCT_Dataset(Dataset):
    def __init__(self, phase, transforms=True):
        self.phase = phase
        self.transforms = transforms
        if phase == 'train':
            self.df = train_test_dict['train']
        else:
            self.df = train_test_dict['test']
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img_path = self.df.loc[idx, 'img_path']
        img = Image.open(img_path).convert('RGB')
        label = self.df.loc[idx, 'label']
        
        if self.transforms:
            if self.phase == 'train':
                transformer = train_test_transformer['train']
            else:
                transformer = train_test_transformer['test']

            img = transformer(img)
        
        return (img, label)

In [14]:
train_dataset = TCT_Dataset('train')
test_dataset = TCT_Dataset('test')

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [44]:
from torchvision.models import inception_v3
model = inception_v3(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

fc_features = model.fc.in_features
model.fc = torch.nn.Linear(fc_features, CLASSES)
model.aux_logits=False

model.to(DEVICE)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [53]:
optimizer = optim.SGD([{'params':model.fc.parameters(), 'lr':0.004},
                      {'params':model.Mixed_7c.parameters(), 'lr':0.001}])
#                        {'params':model.layer4.parameters(),'lr':0.00001}
criterion =  nn.CrossEntropyLoss().cuda()

In [42]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
#         output = torch.tensor(output)
        output = output.to(device)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    print ('Train Epoch: {}\t Loss: {:.6f}\n'.format(epoch,loss.item()))

    
def test(model, device, test_loader, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i,data in enumerate(test_loader):          
            x,y= data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            test_loss = criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
#     test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(train_test_dict['test'])
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(train_test_dict['test']),
        acc))
    
#     print('-----------------------')
    return acc

In [54]:
max_acc = 0
max_epoch = 0
for epoch in range(EPOCH):
    train(model, DEVICE, train_loader, optimizer, epoch)
    acc = test(model, DEVICE, test_loader, optimizer)
    if acc >= max_acc:
        max_acc = acc
        max_epoch = epoch
    print('current best acc: ', max_acc)
    print('-----------------------')
print('Result: at {} epoch, achieved best acc: {:.0f}'.format(max_epoch, max_acc))

Train Epoch: 0	 Loss: 0.552790


Test set: Average loss: 0.6428, Accuracy: 559/692 (81%)

current best acc:  80.78034682080924
-----------------------
Train Epoch: 1	 Loss: 0.558916


Test set: Average loss: 0.4483, Accuracy: 550/692 (79%)

current best acc:  80.78034682080924
-----------------------
Train Epoch: 2	 Loss: 0.506376


Test set: Average loss: 0.4809, Accuracy: 553/692 (80%)

current best acc:  80.78034682080924
-----------------------
Train Epoch: 3	 Loss: 0.537170


Test set: Average loss: 0.5734, Accuracy: 554/692 (80%)

current best acc:  80.78034682080924
-----------------------
Train Epoch: 4	 Loss: 0.638581


Test set: Average loss: 0.6833, Accuracy: 558/692 (81%)

current best acc:  80.78034682080924
-----------------------
Train Epoch: 5	 Loss: 0.584914


Test set: Average loss: 0.4998, Accuracy: 558/692 (81%)

current best acc:  80.78034682080924
-----------------------
Train Epoch: 6	 Loss: 0.546624


Test set: Average loss: 0.5078, Accuracy: 557/692 (80%)

curr

Train Epoch: 54	 Loss: 0.584177


Test set: Average loss: 0.5064, Accuracy: 555/692 (80%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 55	 Loss: 0.485601


Test set: Average loss: 0.4809, Accuracy: 562/692 (81%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 56	 Loss: 0.644733


Test set: Average loss: 0.5071, Accuracy: 563/692 (81%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 57	 Loss: 0.628785


Test set: Average loss: 0.4574, Accuracy: 557/692 (80%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 58	 Loss: 0.581467


Test set: Average loss: 0.4702, Accuracy: 560/692 (81%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 59	 Loss: 0.577271


Test set: Average loss: 0.4352, Accuracy: 558/692 (81%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 60	 Loss: 0.634538


Test set: Average loss: 0.4878, Accuracy: 558/692 (81%

Train Epoch: 108	 Loss: 0.679663


Test set: Average loss: 0.4088, Accuracy: 554/692 (80%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 109	 Loss: 0.688416


Test set: Average loss: 0.5108, Accuracy: 556/692 (80%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 110	 Loss: 0.423965


Test set: Average loss: 0.5223, Accuracy: 555/692 (80%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 111	 Loss: 0.659401


Test set: Average loss: 0.5539, Accuracy: 555/692 (80%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 112	 Loss: 0.538301


Test set: Average loss: 0.5253, Accuracy: 560/692 (81%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 113	 Loss: 0.457414


Test set: Average loss: 0.5163, Accuracy: 560/692 (81%)

current best acc:  81.35838150289017
-----------------------
Train Epoch: 114	 Loss: 0.724541


Test set: Average loss: 0.4851, Accuracy: 557/6

Train Epoch: 162	 Loss: 0.455403


Test set: Average loss: 0.4116, Accuracy: 562/692 (81%)

current best acc:  81.5028901734104
-----------------------
Train Epoch: 163	 Loss: 0.735122


Test set: Average loss: 0.5304, Accuracy: 558/692 (81%)

current best acc:  81.5028901734104
-----------------------
Train Epoch: 164	 Loss: 0.593845


Test set: Average loss: 0.4580, Accuracy: 558/692 (81%)

current best acc:  81.5028901734104
-----------------------
Train Epoch: 165	 Loss: 0.671317


Test set: Average loss: 0.5610, Accuracy: 560/692 (81%)

current best acc:  81.5028901734104
-----------------------
Train Epoch: 166	 Loss: 0.636722


Test set: Average loss: 0.4059, Accuracy: 558/692 (81%)

current best acc:  81.5028901734104
-----------------------
Train Epoch: 167	 Loss: 0.517622


Test set: Average loss: 0.6283, Accuracy: 559/692 (81%)

current best acc:  81.5028901734104
-----------------------
Train Epoch: 168	 Loss: 0.497276


Test set: Average loss: 0.3265, Accuracy: 563/692 (81

In [3]:
help(torch.save)

Help on function save in module torch.serialization:

save(obj, f, pickle_module=<module 'pickle' from '/home/yaoms/anaconda3/envs/PytorchEnv/lib/python3.5/pickle.py'>, pickle_protocol=2)
    Saves an object to a disk file.
    
    See also: :ref:`recommend-saving-models`
    
    Args:
        obj: saved object
        f: a file-like object (has to implement write and flush) or a string
           containing a file name
        pickle_module: module used for pickling metadata and objects
        pickle_protocol: can be specified to override the default protocol
    
    .. warning::
        If you are using Python 2, torch.save does NOT support StringIO.StringIO
        as a valid file-like object. This is because the write method should return
        the number of bytes written; StringIO.write() does not do this.
    
        Please use something like io.BytesIO instead.
    
    Example:
        >>> # Save to file
        >>> x = torch.tensor([0, 1, 2, 3, 4])
        >>> torch.sav